# Prorrata ERNC
Este programa tiene por objetivo realizar el re-calculo de curtailment para el sistema, en base a metodología propuesta por la NTCyO.

## LECTURA DE DATOS
Los datos deben ser extraidos desde el accdb en potencia neta (no por la potencia, sino por los marginales no truncados). La lista de datos que se deben extraer son:
1. Generación de cada central.
2. Perfil de generación de cada central.
3. Barra asociada a cada central.
4. Costos marginales para cada.
5. Curtailment por central (quizas por barra es suficiente).
6. Potencia máxima.
7. Generación disponible.
8. Estado de operación.

Para la lectura, a modo de determinar la mejor query al sistema, sin tener que lidiar con los problemas de MS Access, se cargan las tablas en DuckDB y se utiliza jupysql para probar SQL.

In [81]:
import polars as pl
import duckdb as duck

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
    inspect
)

path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)
    tables = inspect(prg_engine).get_table_names()

    conn = duck.connect("PCP.duckdb")
    #conn.execute("CREATE SCHEMA IF NOT EXISTS bronze")

    for table in tables:
        print(f"trabajando en tabla: {table}...")
        df = pl.read_database(query=f"SELECT * FROM {table}", connection=prg_engine)
        conn.execute(f"CREATE OR REPLACE TABLE {table} AS SELECT * FROM df")

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    conn.close()
    prg_engine.dispose()

trabajando en tabla: t_attribute...
trabajando en tabla: t_attribute_data...
trabajando en tabla: t_band...
trabajando en tabla: t_category...
trabajando en tabla: t_class...
trabajando en tabla: t_class_group...
trabajando en tabla: t_collection...
trabajando en tabla: t_config...
trabajando en tabla: t_custom_column...
trabajando en tabla: t_data_0...
trabajando en tabla: t_data_1...
trabajando en tabla: t_data_2...
trabajando en tabla: t_data_3...
trabajando en tabla: t_data_4...
trabajando en tabla: t_data_6...
trabajando en tabla: t_data_7...
trabajando en tabla: t_data_current...
trabajando en tabla: t_key...
trabajando en tabla: t_key_index...
trabajando en tabla: t_membership...
trabajando en tabla: t_memo_object...
trabajando en tabla: t_model...
trabajando en tabla: t_object...
trabajando en tabla: t_object_meta...
trabajando en tabla: t_period_0...
trabajando en tabla: t_period_1...
trabajando en tabla: t_period_2...
trabajando en tabla: t_period_3...
trabajando en tabla: t_

## REVISIÓN DUCKDB
Con la data carga en la base de datos, empezamos a mirar como armar la mejor query

In [82]:
# Esto es para carga la extensión y leer la base de datos.
import duckdb

conn_pcp = duckdb.connect("pcp.duckdb")

# load de la extensión para sql
%load_ext sql
%sql conn_pcp --alias duck

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [88]:
# Esto es para cerrar las conexiones, usarlo al terminar de revisar
%sql --close duck
conn.close()

Clasica query para sacar la generación, levemente modificada para sacar las otras propiedades de una, no es necesario tener una query por dato.

In [87]:
%%sql
SELECT 
    t_child.name AS generator,
    t_property.name AS property,
    t_period_0.datetime,
    t_data_0.key_id AS data_key,
    t_data_0.period_id AS data_period,
    t_data_0.value,
FROM ((((((((t_membership
INNER JOIN t_collection ON t_membership.collection_id = t_collection.collection_id)
INNER JOIN t_object AS t_parent ON t_membership.parent_object_id = t_parent.object_id)
INNER JOIN t_object AS t_child ON t_membership.child_object_id = t_child.object_id)
INNER JOIN t_property ON t_collection.collection_id = t_property.collection_id)
INNER JOIN t_key ON t_membership.membership_id = t_key.membership_id AND t_property.property_id = t_key.property_id)
INNER JOIN t_data_0 ON t_key.key_id = t_data_0.key_id)
INNER JOIN t_phase_3 ON t_data_0.period_id = t_phase_3.period_id)
INNER JOIN t_period_0 ON t_phase_3.interval_id = t_period_0.interval_id)
INNER JOIN t_category ON t_child.category_id = t_category.category_id
WHERE t_collection.collection_id = 1 AND t_property.property_id IN (1, 28, 200, 219) AND t_category.category_id IN (95, 96, 99, 100)

Running query in 'duck'

generator,property,datetime,data_key,data_period,value
EL_MAITEN_EO,Max Capacity,2024-01-11 00:00:00,10428,1,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 01:00:00,10428,2,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 02:00:00,10428,3,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 03:00:00,10428,4,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 04:00:00,10428,5,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 05:00:00,10428,6,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 06:00:00,10428,7,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 07:00:00,10428,8,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 08:00:00,10428,9,9.0
EL_MAITEN_EO,Max Capacity,2024-01-11 09:00:00,10428,10,9.0


Similar a la anterior, una query para sacar los datos con marginales negativos, no se necesita el resto.

In [86]:
%%sql
SELECT 
    t_child.name AS node,
    t_period_0.datetime,
    t_data_0.key_id AS data_key,
    t_data_0.period_id AS data_period,
    t_data_0.value AS marginal_cost,
FROM ((((((((t_membership
INNER JOIN t_collection ON t_membership.collection_id = t_collection.collection_id)
INNER JOIN t_object AS t_parent ON t_membership.parent_object_id = t_parent.object_id)
INNER JOIN t_object AS t_child ON t_membership.child_object_id = t_child.object_id)
INNER JOIN t_property ON t_collection.collection_id = t_property.collection_id)
INNER JOIN t_key ON t_membership.membership_id = t_key.membership_id AND t_property.property_id = t_key.property_id)
INNER JOIN t_data_0 ON t_key.key_id = t_data_0.key_id)
INNER JOIN t_phase_3 ON t_data_0.period_id = t_phase_3.period_id)
INNER JOIN t_period_0 ON t_phase_3.interval_id = t_period_0.interval_id)
INNER JOIN t_category ON t_child.category_id = t_category.category_id
WHERE t_collection.collection_id = 245 AND t_property.property_id = 1233 AND t_data_0.value < 0

Running query in 'duck'

node,datetime,data_key,data_period,marginal_cost
ElPenon110,2024-01-11 09:00:00,29602,10,-4.9626358250636
Francisco220,2024-01-11 10:00:00,29637,11,-1.86184524305163
ElPenon110,2024-01-11 11:00:00,29602,12,-0.216977423477753
ElPenon110,2024-01-11 12:00:00,29602,13,-0.263556176073346
ElPenon110,2024-01-11 13:00:00,29602,14,-0.215534871550137
ElPenon110,2024-01-11 14:00:00,29602,15,-0.19682939692686
ElPenon110,2024-01-11 15:00:00,29602,16,-0.183725258261048
ElPenon110,2024-01-11 16:00:00,29602,17,-0.174420067605554
ElPenon110,2024-01-11 17:00:00,29602,18,-0.156235601290092
ElPenon110,2024-01-11 18:00:00,29602,19,-0.289653561310179


Query bonita con CTE para extraer la relación entre barra y generador. Lamentablemente no hay CTE en MSACCESS por lo que se reformula en el .sql

In [85]:
%%sql
WITH node_obj AS (
    SELECT 
        t_object.object_id AS node_id,
        t_object.name AS node,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Node'
), gen_obj AS (
    SELECT 
        t_object.object_id AS gen_id,
        t_object.name AS generator,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Generator' AND t_object.category_id IN (95, 96, 99, 100)
)

SELECT
    node_obj.node,
    gen_obj.generator,
FROM t_membership
INNER JOIN node_obj ON t_membership.child_object_id = node_obj.node_id
INNER JOIN gen_obj ON t_membership.parent_object_id = gen_obj.gen_id
WHERE t_membership.collection_id = 12

Running query in 'duck'

node,generator
Andes220,SOL_DEL_NORTE_ANDES_FV
Angamos220,TALLADO_FV
Arica066,PAMPA_CAMARONES_FV
Cachiyuyal220,PAMPA_SOLAR_NORTE_FV
Capricornio110,CAPRICORNIO_FV
Cardones110,VALLE_SOLAR_OESTE_FV
Cardones220,VALLE_ESCONDIDO_FV
Condores220,WILLKA_FV
CPinto220,SAN_ANDRES_FV
Crucero220,LAS_SALINAS_FV


## Juntando la información
Con el trabajo de armar los SQL, ahora se pasa a solo usar polars para disminuir la necesidad de otra libreria `DuckDB` (por mucho que me guste esta db).

In [40]:
import polars as pl

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
)

# Lectura de SQL Barra-Generador
path_sql_node = Path(r"../poc_prorrataerv/sql/gen_node.sql").absolute()
with open(path_sql_node, "r") as file:
    sql_node = file.read()

# Lectura de SQL con data de generacion
path_sql_gen = Path(r"../poc_prorrataerv/sql/gen_data.sql").absolute()
with open(path_sql_gen, "r") as file:
    sql_gen = file.read()

# lectura de SQL con data de barras con costos marginales menor a 0
path_sql_cmg = Path(r"../poc_prorrataerv/sql/cmg_data.sql").absolute()
with open(path_sql_cmg, "r") as file:
    sql_cmg = file.read()

# Inicio de captura de datos en dataframes
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)

    df_nodes = pl.read_database(query=sql_node, connection=prg_engine)
    df_gen = pl.read_database(query=sql_gen, connection=prg_engine)
    df_cmg = pl.read_database(query=sql_cmg, connection=prg_engine)

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    prg_engine.dispose()


In [41]:
# Lectura de otros datos pmgd
path_pmgd = Path(r"W:/41 Dpto Pronosticos/Vertimiento_ERNC/Lista_PMGDs.xlsx").absolute()
df_pmgd = pl.read_excel(
    source=path_pmgd,
    sheet_name="Hoja1",
    xlsx2csv_options={"skip_empty_lines": True},
    read_csv_options={"new_columns": ["Nombre_CDC","Centrales"]},
)

# lectura de lista de centrales vetadas
path_vetados = Path(r"R:/Aplicaciones/Prorrateo_Vertimiento/Centrales_Vetadas.xlsx").absolute()
df_vetados = pl.read_excel(
    source=path_vetados,
    sheet_name="Hoja1",
    xlsx2csv_options={"skip_empty_lines": True},
    read_csv_options={"new_columns": ["Centrales"]},
)

In [3]:
# Check errores de curtailement por centrales no pintadas en rojo.
(
    df_gen
    .filter(
        pl.col("generator").is_in(
            pl.concat(
                [df_vetados["Centrales"].unique(),
                 df_pmgd["Centrales"].unique()]
            )
        )
    )
    .pivot(
        values="value",
        columns="property",
        index=["generator", "datetime"]
    )
    .filter(
        pl.col("Units Generating") == 1,
        pl.col("Capacity Curtailed") != 0,
    )
    .select(
        pl.exclude("Units Generating")
    )
    .group_by(pl.col("generator").alias("Generator"))
    .agg(pl.col("Capacity Curtailed").sum().alias("Total Capacity Curtailed"))
)

Generator,Total Capacity Curtailed
str,f64
"""USYA_FV""",50.543
"""JOTABECHE_FV""",3.522102
"""PARANAL_FV""",23.154222
"""ARMAZONES_FV""",45.930038
"""SAN_PEDRO_GPG_…",89.5
"""TRAVESIA_FV""",1.982644


In [42]:
# transformación de data para generación, eliminando centrales vetadas y pmgd
# y pivotear la data para tener las columnas de las propiedades
# y filtrar las centrales que no estan generando
df_gen_pivot = (
    df_gen
    .filter(
        ~pl.col("generator").is_in(df_vetados["Centrales"].unique()),
        ~pl.col("generator").is_in(df_pmgd["Centrales"].unique()),
    )
    .pivot(
        values="value",
        columns="property",
        index=["generator", "datetime"]
    )
    .filter(
        pl.col("Units Generating") == 1,
    )
    .select(
        pl.exclude("Units Generating")
    )
)
df_gen_pivot

generator,datetime,Generation,Capacity Curtailed,Max Capacity,Available Capacity
str,datetime[μs],f64,f64,f64,f64
"""ANCOA""",2024-01-11 00:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 01:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 02:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 03:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 04:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 05:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 06:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 07:00:00,23.2,0.0,24.656,23.2
"""ANCOA""",2024-01-11 08:00:00,23.2,0.0,24.656,23.2


In [43]:
def calc_error(df: pl.LazyFrame, original_col: str = "Generation", prorrata_col: str = "Prorrata", over_col: str = "datetime") -> pl.LazyFrame:
    return (
        df
        .with_columns(
            pl.when(pl.col("Prorrata").lt(0))
            .then(pl.col("Prorrata").abs())
            .otherwise(0)
            .alias("Error"),
            pl.when(pl.col("Prorrata").lt(0))
            .then(0)
            .otherwise(pl.col("Prorrata"))
            .alias("Prorrata"),
        )
    )

def check_error(df: pl.LazyFrame, error_col: str = "Error", tol: float = 1e-3) -> bool:
    return df.select(pl.col(error_col).ge(tol).any()).collect().item()

def show_total_error(df: pl.LazyFrame, error_col: str = "Error") -> float:
    return df.select(pl.col(error_col).sum().alias("Total Error")).collect().item()

def calc_prorrata(df: pl.LazyFrame, target_col: str = "Prorrata", error_col: str = "Error", weight_col: str = "Max Capacity", over_col: str = "datetime") -> pl.LazyFrame:
    return (
        df
        .with_columns(
            (pl.col(target_col) - pl.col(error_col).sum().over(over_col) * pl.col(weight_col) / pl.col(weight_col).sum().over(over_col)).alias("Prorrata"),
        )
    )

In [44]:
def process_prorrata(df: pl.LazyFrame, target_col: str = "Prorrata", error_col: str = "Error", weight_col: str = "Max Capacity", over_col: str = "datetime") -> pl.LazyFrame:
    df_processed = calc_prorrata(df,target_col,error_col)
    df_processed = calc_error(df_processed)

    print(check_error(df_processed))
    print(show_total_error(df_processed))

    if check_error(df_processed):
        return process_prorrata(df_processed)
    
    return df_processed

In [45]:
test_data = (
    df_cmg
    .join(df_nodes, on="node", how="inner")
    .join(df_gen_pivot, on=["generator","datetime"], how="inner")
)

In [46]:
test_prorrata_func = process_prorrata(test_data.lazy(),"Available Capacity","Capacity Curtailed")

True
287.62912102288067
True
39.23430898394995
True
6.99900362532531
True
1.3878593732879132
True
0.29011683766575624
True
0.06250819986513116
True
0.013725650134728495
False
0.003051937765409438


In [47]:
test_prorrata_func.collect().describe()

describe,node,datetime,generator,Generation,Capacity Curtailed,Max Capacity,Available Capacity,Prorrata,Error
str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""823""","""823""","""823""",823.0,823.0,823.0,823.0,823.0,823.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,46.685793,9.270782,82.792001,55.956575,46.685797,0.000004
"""std""",null,null,null,65.275302,25.71281,77.776877,64.189668,55.799891,0.000031
"""min""","""AJahuel220""","""2024-01-11 09:…","""ABANICO""",0.0,0.0,0.3,0.003134,0.0,0.0
"""25%""",null,null,null,1.1,0.0,10.5,6.4,4.157271,0.0
"""50%""",null,null,null,14.5,0.0,69.0,37.88,29.26836,0.0
"""75%""",null,null,null,74.68,0.0,115.5,87.3,72.38615,0.0
"""max""","""Tinguiririca15…","""2024-01-11 18:…","""VALLE_ESCONDID…",366.746,164.9,381.0,366.746,322.012205,0.000367


In [48]:
(
    test_prorrata_func
    .sort(by="datetime")
    .group_by("datetime")
    .agg(
        #pl.col("Prorrata_Curt").sum().alias("Total_Curatiled"),
        pl.col("Generation").sum().alias("Total_Gen"),
        pl.col("Prorrata").sum().alias("Total_Gen_Prorrata"),
        #pl.col("Prorrata").min().alias("Min_Gen_Prorrata"),
        #pl.col("Prorrata").filter(pl.col("Prorrata").lt(0)).sum().alias("Sum_Gen_Prorrata"),
        pl.col("Error").sum().alias("Total_Error"),
        (pl.col("Prorrata") - pl.col("Error")).sum().alias("Sum_Prorrata_error"),
        (pl.col("Generation") - (pl.col("Prorrata") - pl.col("Error"))).sum().alias("Test_total"),
    )
).collect()

datetime,Total_Gen,Total_Gen_Prorrata,Total_Error,Sum_Prorrata_error,Test_total
datetime[μs],f64,f64,f64,f64,f64
2024-01-11 09:00:00,5192.164816,5192.164816,1.1714e-8,5192.164816,-2.3297e-8
2024-01-11 10:00:00,1267.639286,1267.639286,5.7114e-9,1267.639286,-4.2596e-9
2024-01-11 11:00:00,3819.634563,3819.637407,0.002844,3819.634563,-8.4385e-9
2024-01-11 12:00:00,3940.779408,3940.779608,0.0002,3940.779408,2.7451e-9
2024-01-11 13:00:00,3939.977646,3939.977655,0.000008,3939.977646,-7.7408e-10
2024-01-11 14:00:00,3917.010157,3917.010157,2.4655e-7,3917.010157,4.2945e-9
2024-01-11 15:00:00,4154.263024,4154.263024,1.8354e-10,4154.263024,-3.5326e-9
2024-01-11 16:00:00,4034.743655,4034.743655,3.2390e-12,4034.743655,3.5579e-9
2024-01-11 17:00:00,4063.363033,4063.363033,1.3848e-12,4063.363033,2.5116e-9


In [85]:
test_prorrata_func.collect()

node,datetime,generator,Generation,Capacity Curtailed,Max Capacity,Available Capacity,Prorrata,Error
str,datetime[μs],str,f64,f64,f64,f64,f64,f64
"""Linares154""",2024-01-11 09:00:00,"""ANCOA""",23.2,0.0,24.656,23.2,23.008707,0.0
"""AJahuel220""",2024-01-11 09:00:00,"""CARENA""",2.75,0.0,10.0,2.75,2.672415,0.0
"""Polpaico220""",2024-01-11 09:00:00,"""CHACABUQUITO""",17.4,0.0,25.7,17.4,17.200607,0.0
"""Arica066""",2024-01-11 15:00:00,"""CHAPIQUINA""",3.5,0.0,10.14,3.5,1.957316,0.0
"""Arica066""",2024-01-11 16:00:00,"""CHAPIQUINA""",3.5,0.0,10.14,3.5,1.766191,0.0
"""Arica066""",2024-01-11 17:00:00,"""CHAPIQUINA""",3.5,0.0,10.14,3.5,2.017652,0.0
"""Arica066""",2024-01-11 18:00:00,"""CHAPIQUINA""",3.5,0.0,10.14,3.5,3.013363,0.0
"""Itahue154""",2024-01-11 09:00:00,"""CONVENTO_VIEJO…",14.5,0.0,19.4,14.5,14.349485,0.0
"""Tinguiririca15…",2024-01-11 09:00:00,"""EL_PASO""",57.0,0.0,60.0,57.0,56.534491,0.0


In [86]:
df_gen

generator,property,datetime,data_key,data_period,value
str,str,datetime[μs],i64,i64,f64
"""ALFALFAL""","""Generation""",2024-01-11 00:00:00,519,1,136.62
"""ALFALFAL""","""Generation""",2024-01-11 01:00:00,519,2,139.55
"""ALFALFAL""","""Generation""",2024-01-11 02:00:00,519,3,133.27
"""ALFALFAL""","""Generation""",2024-01-11 03:00:00,519,4,136.26
"""ALFALFAL""","""Generation""",2024-01-11 04:00:00,519,5,135.92
"""ALFALFAL""","""Generation""",2024-01-11 05:00:00,519,6,136.97
"""ALFALFAL""","""Generation""",2024-01-11 06:00:00,519,7,137.0
"""ALFALFAL""","""Generation""",2024-01-11 07:00:00,519,8,136.67
"""ALFALFAL""","""Generation""",2024-01-11 08:00:00,519,9,135.67


In [100]:
data_to_update = (
    test_prorrata_func
    .join(
        df_gen.filter(pl.col("property") == "Generation").lazy(),
        on=["generator","datetime"],
        how="inner"
    )
    .select(
        #pl.col("generator"),
        #pl.col("datetime"),
        #pl.col("Generation"),
        #pl.col("value").alias("Generation_Ori"),
        pl.col("data_key").alias("key_id"),
        pl.col("data_period").alias("period_id"),
        pl.col("Prorrata").alias("value"),
    )
    .sort(by=["key_id","period_id"])
    .collect()
)
data_to_update

key_id,period_id,value
i64,i64,f64
547,10,23.008707
589,10,2.672415
617,10,17.200607
645,16,1.957316
645,17,1.766191
645,18,2.017652
645,19,3.013363
687,10,14.349485
743,10,56.534491


In [112]:
test_dict = data_to_update.head().to_dicts()
for row in test_dict:
    print(row)

{'key_id': 547, 'period_id': 10, 'value': 23.008706735285774}
{'key_id': 589, 'period_id': 10, 'value': 2.6724151262515297}
{'key_id': 617, 'period_id': 10, 'value': 17.200606874466427}
{'key_id': 645, 'period_id': 16, 'value': 1.9573155816215664}
{'key_id': 645, 'period_id': 17, 'value': 1.7661906158738894}


In [7]:
from sqlalchemy import (
    engine,
    create_engine,
    inspect,
)
from sqlalchemy.sql.expression import (
    update,
    table,
    column,
)

from pathlib import Path

import polars as pl

df_data = pl.read_csv("../t_data_0.csv")

t_data_0 = table("t_data_0", column("key_id"), column("period_id"), column("value"))
#stmt = update(t_data_0).where(t_data_0.c.key_id == 1).values(value=0.0)

# Inicio de captura de datos en dataframes
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

prg_engine = create_engine(connection_url)
tables = inspect(prg_engine).get_table_names()

with prg_engine.begin() as conn:
    for row in df_data.head().to_dicts():
        stmt = (
            t_data_0.update()
            .where(t_data_0.c.key_id == row.get("key_id"))
            .where(t_data_0.c.period_id == row.get("period_id"))
            .values(value=row.get("value"))
        )
        restuls = conn.execute(stmt)
        print(restuls.)
    #conn.commit()


0
0
0
0
0


In [50]:
import pyodbc
import polars as pl
from pathlib import Path

df_data = pl.read_csv("../t_data_0.csv")
to_update = df_data.head().to_dicts()


path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)

cnxn = pyodbc.connect(connection_string)
crsr = cnxn.cursor()

#sql = "SELECT TOP 10 * FROM t_data_0 "
#update_sql = r"UPDATE t_data_0 SET t_data_0.value = ? WHERE t_data_0.key_id = ? AND t_data_0.period_id = ?;"
#params = [(data['value'],data['key_id'],data['period_id']) for data in to_update]
#crsr.execute(update_sql, params)
#crsr.commit()
#cnxn.close()

try:
    cnxn.autocommit = False
    params = params = [(data['value'],data['key_id'],data['period_id']) for data in to_update]
    crsr.executemany("UPDATE t_data_0 SET t_data_0.value = ? WHERE t_data_0.key_id = ? AND t_data_0.period_id = ?;", params)
except pyodbc.DatabaseError as err:
    cnxn.rollback()
else:
    cnxn.commit()
finally:
    cnxn.close()

In [48]:
[(data['value'],data['key_id'],data['period_id']) for data in to_update]

[(22.9, 547, 9),
 (19.40015313, 547, 10),
 (18.22120423, 547, 11),
 (17.31699723, 547, 12),
 (15.13053132, 547, 13)]

In [46]:
df_data.head().to_dict(as_series=False)['key_id']

[547, 547, 547, 547, 547]

In [2]:
from pathlib import Path
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_string

'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=c:/Users/felipe.bastidas/PyProyectos/PoC_ProrrataERV/dev/../data/Model PRGdia_Full_Definitivo Solution_neto.accdb;ExtendedAnsiSQL=1;'

## Otroas Pruebas
Revisando detalles del manejo de rutas.

In [39]:
from pathlib import Path

path_prg = r"C:\Users\felipe.bastidas\PyProyectos\prg_20240128"
PATH_ACCDB = "Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb"

path_final = Path(path_prg)

if path_final.exists():
    path = path_final
else:
    raise ValueError(f"Path: {path_prg} does not exists.")

temp_path = path / PATH_ACCDB

if temp_path.exists():
    path = temp_path
else:
    raise ValueError(f"Path: {temp_path} does not exists.")

In [37]:
otra_ruta = "Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb"
path / path_accdbs

WindowsPath('C:/Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb')

In [36]:
temp_path = path / path_accdbs
temp_path

WindowsPath('C:/Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb')

In [3]:
import argparse

parser = argparse.ArgumentParser(description="draw basic graphs on terminal")
parser.add_argument(
    "filename",
    nargs="?",
    default="-",
    help="data file name (comma or space separated). Defaults to stdin.",
)
parser.add_argument("--title", help="Title of graph")
parser.add_argument(
    "--width", type=int, default=50, help="width of graph in characters default:50"
)
parser.add_argument("--format", default="{:<5.2f}", help="format specifier to use.")
parser.add_argument(
    "--suffix", default="", help="string to add as a suffix to all data points."
)
parser.add_argument(
    "--no-labels", action="store_true", help="Do not print the label column"
)
parser.add_argument(
    "--no-values", action="store_true", help="Do not print the values at end"
)
parser.add_argument(
    "--space-between",
    action="store_true",
    help="Print a new line after every field",
)
parser.add_argument("--color", nargs="*", help="Graph bar color( s )")
parser.add_argument("--vertical", action="store_true", help="Vertical graph")
parser.add_argument("--stacked", action="store_true", help="Stacked bar graph")
parser.add_argument("--histogram", action="store_true", help="Histogram")
parser.add_argument("--bins", default=5, type=int, help="Bins of Histogram")
parser.add_argument(
    "--different-scale",
    action="store_true",
    help="Categories have different scales.",
)
parser.add_argument(
    "--calendar", action="store_true", help="Calendar Heatmap chart"
)
parser.add_argument("--start-dt", help="Start date for Calendar chart")
parser.add_argument(
    "--custom-tick", default="", help="Custom tick mark, emoji approved"
)
parser.add_argument(
    "--delim", default="", help="Custom delimiter, default , or space"
)
parser.add_argument(
    "--verbose", action="store_true", help="Verbose output, helpful for debugging"
)
parser.add_argument(
    "--label-before",
    action="store_true",
    default=False,
    help="Display the values before the bars",
)
parser.add_argument(
    "--version", action="store_true", help="Display version and exit"
)
args = vars(parser.parse_args())

In [8]:
import re
args["color"] = ["blue,red"]
args["color"][0]
colorargs = re.findall(r"[a-z]+", args["color"][0])

NameError: name 'args' is not defined

In [44]:
import polars as pl

data = pl.read_csv(
    "results.csv",
    has_header=False,
    new_columns=["datatime","Generacion","Prorrata"],
    dtypes={"datatime": pl.Datetime, "Generacion": pl.Float64, "Prorrata": pl.Float64},
)
data.select(pl.col("datatime").dt.to_string("%Y/%m/%d %H:%M"))

datatime
str
"""2024/01/28 08:…"
"""2024/01/28 09:…"
"""2024/01/28 10:…"
"""2024/01/28 11:…"
"""2024/01/28 12:…"
"""2024/01/28 13:…"
"""2024/01/28 14:…"
"""2024/01/28 15:…"
"""2024/01/28 16:…"


In [48]:
from datetime import datetime
for row in data.iter_rows():
    print(row[0].strftime('%Y-%m-%d %H:%M'),str(row[1]),row[2]) 

2024-01-28 08:00 3685.7746478618046 3685.774647894004
2024-01-28 09:00 3826.365923642415 3826.366001804971
2024-01-28 10:00 4029.8190949281425 4029.8220470943434
2024-01-28 11:00 4056.4111663333483 4056.4183488257495
2024-01-28 12:00 3692.5157544370054 3692.5264510220723
2024-01-28 13:00 3790.5156957481945 3790.5211362573937
2024-01-28 14:00 4044.9561936349564 4044.958635853661
2024-01-28 15:00 4166.019679341192 4166.021516020728
2024-01-28 16:00 3805.5742595074153 3805.5755061459286
2024-01-28 17:00 4157.587291915179 4157.587461670527
2024-01-28 18:00 4143.025591253948 4143.025591639975


In [14]:
from termgraph import termgraph as tg
from colorama import init


init()
args_default = tg.init_args()
args_default["title"] = "Resultados prorrata ERV"
args_default["color"] = ["blue,red"]
args_default["filename"] = "results.csv"
# Agregar categorias para el gráfico.
##linea_texto = '@ Generacion, Prorrata'
#with open(args_default["filename"], 'a', encoding='utf-8') as archivo:
#    archivo.write(linea_texto)
_, labels, data, colors = tg.read_data(args_default)
colorargs = re.findall(r"[a-z]+", args_default["color"][0])
catergories = ["Generación","Prorrata"]
tg.print_categories(catergories, colorargs)
tg.chart(colors, data, args_default, labels)


# Resultados prorrata ERV

luemluem▇ Generación  edmedm▇ Prorrata  


2024-01-28T08:00:00.000000: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\AppData\Roaming\jupyter\runtime\kernel-v2-42968HapOFMtHRmjv.jsonK
                            ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\AppData\Roaming\jupyter\runtime\kernel-v2-42968HapOFMtHRmjv.jsonK
2024-01-28T09:00:00.000000: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\AppData\Roaming\jupyter\runtime\kernel-v2-42968HapOFMtHRmjv.jsonK
                            ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\AppData\Roaming\jupyter\runtime\kernel-v2-42968HapOFMtHRmjv.jsonK
2024-01-28T10:00:00.000000: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\AppData\Roaming\jupyter\runtime\kernel-v2-42968HapOFMtHRmjv.jsonK
                            ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ c:\Users\felipe.bastidas\A

In [6]:
categories

NameError: name 'categories' is not defined

In [13]:
from pathlib import Path

path_prg = Path(r'C:\Users\felipe.bastidas\PyProyectos\prg_20240128\Datos\Model PRGdia_Full_Definitivo Solution')


temp_path = Path(path_prg).parent.parent
path_prg = temp_path.as_posix()

In [14]:
path_prg

'C:/Users/felipe.bastidas/PyProyectos/prg_20240128'